In [1]:
%reload_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import pickle
import DCM

In [2]:
df_psm = pd.read_stata('preference survey module/data/MainSample_Replication.dta')

# Recover answers to each question
# 1 - choosing lottery (A) - higher sure payoff
# 0 - choosing sure payoff (B) - lower sure payoff
# j: layer
# i: node

j_max = 4

for j in range(j_max+1):
    for i in range(2**(j_max-j)):

        col_name = 'layer_' + str(j_max-j+1) + '_q' +str(i+1)

        conditions = [
            df_psm['v_110'].between((2*i)*2**j+1, (2*i+1)*2**j),   
            df_psm['v_110'].between((2*i+1)*2**j+1, (2*i+2)*2**j) 
        ]

        choice_labels = [0, 1]

        new_col = np.select(conditions, choice_labels, default=np.nan)
        
        df_psm = pd.concat([df_psm, pd.DataFrame({col_name: new_col})], axis=1)

# Melt the data
df_psm = df_psm.reset_index(names='pid')

q_columns = [col for col in df_psm.columns if 'layer_' in col]

df_melted = df_psm.melt(id_vars='pid', value_vars=q_columns, 
                        var_name='q_risk', value_name='choice').dropna()

def sure_pay(s):
    parts = s.strip().split('_')

    layer_number = j_max - int(parts[1]) + 1 
    q_number = int(parts[2][1:])

    return 2**(layer_number) + (q_number - 1)*2**(layer_number+1)

df_melted['x'] = df_melted['q_risk'].apply(sure_pay)
df_melted['c'] = 30
df_melted['layer'] = df_melted['q_risk'].apply(lambda x: x.strip().split('_')[1])

# Pivot the data
df_wide = df_melted[['pid','x','choice','layer']].pivot(index='pid', columns='layer', values=['x', 'choice'])

df_wide.columns = [f"layer_{layer}_{col}" for col, layer in df_wide.columns]
df_wide = df_wide.reset_index()

# Merge with the original levels
df_psm = pd.merge(right=df_psm[['pid','v_110']].dropna(),left=df_wide,on='pid')

In [3]:
df_psm

,pid,layer_1_x,layer_2_x,layer_3_x,layer_4_x,layer_5_x,layer_1_choice,layer_2_choice,layer_3_choice,layer_4_choice,layer_5_choice,v_110
0,0,16.0,8.0,4.0,6.0,7.0,0.0,0.0,1.0,1.0,1.0,8.0
1,1,16.0,8.0,4.0,6.0,5.0,0.0,0.0,1.0,0.0,1.0,6.0
2,2,16.0,8.0,12.0,14.0,13.0,0.0,1.0,1.0,0.0,0.0,13.0
3,3,16.0,24.0,28.0,30.0,29.0,1.0,1.0,1.0,0.0,0.0,29.0
4,4,16.0,8.0,12.0,10.0,9.0,0.0,1.0,0.0,0.0,1.0,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...
389,404,16.0,8.0,4.0,6.0,5.0,0.0,0.0,1.0,0.0,1.0,6.0
390,405,16.0,24.0,28.0,26.0,27.0,1.0,1.0,0.0,1.0,0.0,27.0
391,406,16.0,8.0,12.0,10.0,11.0,0.0,1.0,0.0,1.0,1.0,12.0
392,407,16.0,8.0,12.0,14.0,15.0,0.0,1.0,1.0,1.0,0.0,15.0


In [4]:
model = DCM.mixedModel(data=df_psm,
                      x = ['layer_1_x', 'layer_2_x', 'layer_3_x', 'layer_4_x', 'layer_5_x'],
                      choice = ['layer_1_choice', 'layer_2_choice', 'layer_3_choice', 'layer_4_choice', 'layer_5_choice'],
                      fixed_args={'c':30,'delta':0.5})


In [117]:
model.set_init_param(latent_keys=['riskCoef','temp'])

In [249]:
model.set_init_param(init_params={'mean_risk':2,'mean_temp':1,'rho':0,'std_risk':0.1,'std_temp':0.1},var_keys=['riskCoef','temp'])
model.obj_msl([2,1,0,0.1,0.1])

np.float64(1371.7158352671909)

In [150]:
prob = model.postProblatent(latent_class= {'riskCoef':[2.0,1.5,3], 'temp':[2,1,4]}, latent_share= [0.5,0.3,0.2])

In [152]:
prob.sum(axis=0)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1.

In [53]:
model.data[model.choice].values[:,1]

array([0., 0., 1., 1., 1., 1., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1., 0.,
       1., 1., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 0., 1., 1.,
       0., 1., 0., 0., 0., 1., 1., 0., 0., 1., 1., 0., 1., 1., 1., 0., 0.,
       1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 1.,
       0., 0., 1., 0., 1., 0., 1., 1., 1., 1., 0., 0., 0., 0., 0., 1., 0.,
       0., 1., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1.,
       0., 1., 1., 0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 1., 1., 1., 1.,
       0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 0., 0., 1., 0., 1.,
       1., 0., 0., 1., 1., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 0., 1.,
       0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 0., 1., 0., 1.,
       0., 1., 0., 1., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 1., 0.,
       1., 1., 1., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0., 1., 0., 1., 1.,
       0., 1., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1., 0., 0., 1., 1.,
       1., 1., 0., 1., 0.

In [ ]:
with open("msl_result.pkl", "wb") as f:
    pickle.dump(model.msl_result, f)

In [5]:
model_em = DCM.EMmethod(data=df_melted,
                      x = 'x',
                      choice = 'choice',
                      fixed_args={'c':30,'delta':0.5})

In [231]:
model_iid = msl.choiceMSL(data=df_melted,
                      x = 'x',
                      choice = 'choice',
                      fixed_args={'c':30,'delta':0.5})

In [243]:
model_iid.set_init_param(init_params={'riskCoef':60,'temp':10})
model_iid.fit_param()

  message: CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
  success: True
   status: 0
      fun: 1365.500187888806
        x: [ 6.000e+01  7.000e+01]
      nit: 3
      jac: [ 0.000e+00  0.000e+00]
     nfev: 18
     njev: 6
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>

In [151]:
model_iid.result.x

array([68.28912937, 80.80581961])